<a href="https://colab.research.google.com/github/plucyvrz/WebUI/blob/master/StableDiffusionUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Original Script by SegioDev**
## Instruction
Di cell berikutnya ...
1. Periksa username dan password untuk menggunakan WebUI.
2. Ceklis model dan extension yang ingin di gunakan, saat ini ada 30 Models yang berbeda.
4. Klik tombol play untuk run colab (Colab free 5-6 Menit untuk pertama kali run, dan 2 menit untuk re-run jika terjadi kesalahan/crash).
5. Klik link `gradio.live` atau  `ngrok.io` di log untuk lanjut menggunakan AUTOMATIC1111.

##  ⠀
## ngrok (Optional)
Gunakan [ngrok](https://ngrok.com/) jika mengalami tombol yang kurang responsif setelah generate gambar.
  1. Buka https://ngrok.com/
  2. Daftar akun
  3. Verifikasi Email
  4. Copy auth token dari https://dashboard.ngrok.com/get-started/your-authtoken dan pastekan di column ngrok token di colab.

##  ⠀
## Update (Official)
- 5/6/2023 Update controlnet v1.1
- 5/6/2023 Save VAEs in google drive
- 5/5/2023 Added Regional Prompter extension
- 4/27/2023 Fix torch, torchvision version mismatch
- 4/19/2023 Add Deforum, fixed save everything


##  ⠀
## Update (non-official)
- 4/20/2023 Add more Extension & Fix Theme
- 4/8/2023 Fix cache.json & add user.css
- 3/13/2023 Linked to Google Drive



In [ ]:
from IPython.core import extensions
import os, subprocess, time, glob
#@title <font size="6" color="orange">Stable diffusion</font>
#@markdown #### If there is an error or stopped, just re-run, it took 30-120 seconds depending on your account.
#@markdown ##  ⠀
#@markdown #### Follow the URL link **`https://xxxxxxx.gradio.live`** to launch the app.
#@markdown ##### (**`https://xxxxx.ngrok.io`** if use ngrok.)
output_path = 'WebUI' #@param {type:"string"}
username='a' #@param {type:"string"}
password='a' #@param {type:"string"}
NGROK ='5xzL82S2UdECLhvLUtQxv_oeACqyP6M3Py1ABwa1Tv' #@param {type: "string"}
#@markdown ##  ⠀
#@markdown Save images, settings, Lora models, embeddings, load models in Google Drive.

Save_In_Google_Drive = 'Small models, images and settings (Recommended)' #@param ['Everything', 'Small models, images and settings (Recommended)', "Nothing"]
class GDriveSaveMode:
    Everything = 1
    Models_only = 2
    Nothing = 3
if Save_In_Google_Drive == 'Small models, images and settings (Recommended)':
  gMode = GDriveSaveMode.Models_only
elif Save_In_Google_Drive == 'Everything':
  gMode = GDriveSaveMode.Everything
elif Save_In_Google_Drive == 'Nothing':
  gMode = GDriveSaveMode.Nothing
else:
  raise ValueError("Save_In_Google_Drive value invalid.")

Clear_Log = True #@param{type:'boolean'}

#@markdown  ⠀
#@markdown ### Main Models:
Kenshi_01 = True #@param{type: "boolean"}

#@markdown  ⠀
#@markdown ### Art Models:
wlop_1 = False #@param{type: "boolean"}
AbyssalWlop = False #@param{type: "boolean"}
ArtModel = False #@param{type: "boolean"}
nixeu_any = False #@param{type: "boolean"}
nixeu_f = False #@param{type: "boolean"}
wlop_nixeu_robutts = False #@param{type: "boolean"}

#@markdown  ⠀
#@markdown ### Anime Models:
doll774 = False #@param{type: "boolean"}
corneos7thHeavenMix_v2 = False #@param{type: "boolean"}
waifuDiffusion_v13Fp32 = False #@param{type: "boolean"}

#@markdown  ⠀
#@markdown ### Realistic Models:
neverendingDream = False #@param{type: "boolean"}
chilloutmix = False #@param{type: "boolean"}
perfectWorld = False #@param{type: "boolean"}
sunshinemix = False #@param{type: "boolean"}
Basil_mix_fixed = False #@param{type: "boolean"}
povSkinTextureR34 = False #@param{type: "boolean"}
povSkinTexture = False #@param{type: "boolean"}
genericfemalemix = False #@param{type: "boolean"}

#@markdown  ⠀
#@markdown ### v1 models:
v1_4_model = False #@param{type: "boolean"}
v1_5_model = False #@param{type: "boolean"}
v1_5_inpainting_model = False #@param{type: "boolean"}
F222_model = False #@param{type: "boolean"}
DreamShaper_model = False #@param{type: "boolean"}
OpenJourney_model = False #@param{type: "boolean"}
Anything_v3_model = False #@param{type: "boolean"}
Inkpunk_Diffusion_model = False #@param{type: "boolean"}
instruct_pix2pix_model = False #@param{type: "boolean"}
#@markdown  ⠀
#@markdown ### v2 models:
v2_1_768_model = False #@param{type: "boolean"}
v2_1_512_model = False #@param{type: "boolean"}
v2_depth_model = False #@param{type: "boolean"}

#@markdown  ⠀
#@markdown ### Extension(s):
AdditionalNetwork = True #@param{type: "boolean"}
infiniteZoom = False #@param{type: "boolean"}
Deforum = True #@param{type: "boolean"}
ControlNet = True #@param{type: "boolean"}
TagComplete = True #@param{type: "boolean"}
imageBrowser = True #@param{type: "boolean"}
darkTheme = True #@param{type: "boolean"}
CivitAIBrowser = False #@param{type: "boolean"}
RatioHelper = True #@param{type: "boolean"}
Regional_Prompter = True #@param{type: "boolean"}



#@markdown  ⠀
#@markdown ### Install models from URL (separate them with comma).
Model_from_URL = '' #@param {type: "string"}
Save_model_in_Google_Drive = True #@param{type: "boolean"}
#@markdown (Tersedia di RUN selanjutnya)



if gMode != GDriveSaveMode.Nothing:
  # connect to google drive
  from google.colab import drive
  drive.mount('/content/drive') 
  output_path = '/content/drive/MyDrive/' + output_path
  if gMode == GDriveSaveMode.Everything:
    # install A1111 in AI_PIC
    root = output_path 
  elif gMode == GDriveSaveMode.Models_only:
    # install A1111 in Colab
    root = '/content/' 
    # make necessary directories if not exists
    !mkdir -p {output_path}/outputs
    !mkdir -p {output_path}/models
    !mkdir -p {output_path}/ESRGAN
    !mkdir -p {output_path}/hypernetworks
  else:
    raise ValueError("Unexpected gMode: %s"%gMode)
else:
  # Don't connect to google drive
  output_path = '/content/' + output_path
  root = '/content/' 


def clear():
    from IPython.display import clear_output; return clear_output()

def fetch_bytes(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        from urllib.request import urlopen 
        return urlopen(url_or_path) 
    return open(url_or_path, 'r')

def packages():
    import sys, subprocess
    return [r.decode().split('==')[0] for r in subprocess.check_output([sys.executable, '-m', 'pip', 'freeze']).split()]

def downloadModel(url):
  if 'huggingface.co' in url:
    filename = url.split('/')[-1]
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}  -o {filename}
  else:
    # civitai
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {url}

def download_models():
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  print('⏳ Downloading models ...')
  if v1_4_model:
    downloadModel('https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt')
  if v1_5_model:  
    downloadModel('https://huggingface.co/ckpt/sd15/resolve/main/v1-5-pruned-emaonly.ckpt')
  if v1_5_inpainting_model:  
    downloadModel('https://huggingface.co/runwayml/stable-diffusion-inpainting/resolve/main/sd-v1-5-inpainting.ckpt')
  if F222_model:
    downloadModel('https://huggingface.co/acheong08/f222/resolve/main/f222.ckpt')
  if DreamShaper_model:
    downloadModel('https://huggingface.co/Lykon/DreamShaper/resolve/main/Dreamshaper_3.32_baked_vae_clip_fix_half.ckpt')
  if OpenJourney_model:
    downloadModel('https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt')
  if Anything_v3_model:
    downloadModel('https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors')
  if Inkpunk_Diffusion_model:
    downloadModel('https://huggingface.co/Envvi/Inkpunk-Diffusion/resolve/main/Inkpunk-Diffusion-v2.ckpt')
  if instruct_pix2pix_model:
    downloadModel('https://huggingface.co/timbrooks/instruct-pix2pix/resolve/main/instruct-pix2pix-00-22000.ckpt')

  if v2_1_768_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/v2-1_768-ema-pruned.yaml

  if v2_1_512_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/v2-1_512-ema-pruned.yaml

  if v2_depth_model:
    downloadModel('https://huggingface.co/stabilityai/stable-diffusion-2-depth/resolve/main/512-depth-ema.ckpt')
    !wget -nc https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-midas-inference.yaml -O {root}/stable-diffusion-webui/models/Stable-diffusion/512-depth-ema.yaml

  if Kenshi_01:
    downloadModel('https://civitai.com/api/download/models/6792')
  
  if wlop_1:
    downloadModel('https://civitai.com/api/download/models/3266')
  if AbyssalWlop:
    downloadModel('https://huggingface.co/SirVeggie/wlop/resolve/main/AbyssalWlop.ckpt')
  if ArtModel:
    downloadModel('https://huggingface.co/Nerfgun3/ArtModel/resolve/main/ArtModel.ckpt')
  if nixeu_any:
    downloadModel('https://huggingface.co/SirVeggie/nixeu/resolve/main/nixeu-any.ckpt')
  if nixeu_f:
    downloadModel('https://huggingface.co/SirVeggie/nixeu/resolve/main/nixeu-f.ckpt')
  if wlop_nixeu_robutts:
    downloadModel('https://huggingface.co/SirVeggie/wlop-nixeu-robutts/resolve/main/wlop-nixeu-robutts.safetensors')
  
  if doll774:
    downloadModel('https://huggingface.co/doll774/doll774/resolve/main/doll774.ckpt')
  if corneos7thHeavenMix_v2:
    downloadModel('https://huggingface.co/dwarfbum/corneos7thHeavenMix_v2/resolve/main/corneos7thHeavenMix_v2.safetensors')
  if waifuDiffusion_v13Fp32:
    downloadModel('https://civitai.com/api/download/models/121')

  if neverendingDream:
    downloadModel('https://civitai.com/api/download/models/11925')
  if chilloutmix:
    downloadModel('https://civitai.com/api/download/models/11745')
  if perfectWorld:
    downloadModel('https://civitai.com/api/download/models/19084')
  if sunshinemix:
    downloadModel('https://civitai.com/api/download/models/13510')
  if Basil_mix_fixed:
    downloadModel('https://huggingface.co/nuigurumi/basil_mix/resolve/main/Basil_mix_fixed.safetensors')
  if povSkinTextureR34:
    downloadModel('https://civitai.com/api/download/models/5079')
  if povSkinTexture:
    downloadModel('https://civitai.com/api/download/models/5097')
  if genericfemalemix:
    downloadModel('https://civitai.com/api/download/models/5701')

  if Model_from_URL:
      for m in Model_from_URL.split(','):
        %cd {root}/stable-diffusion-webui/models/Stable-diffusion
        downloadModel(m)
        if Save_model_in_Google_Drive and gMode == GDriveSaveMode.Models_only:
          %cd {output_path}/models
          downloadModel(m)

  # download VAEs
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o kl-f8-anime2.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-ema-original/resolve/main/vae-ft-ema-560000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-ema-560000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d {root}/stable-diffusion-webui/models/VAE/ -o vae-ft-mse-840000-ema-pruned.ckpt
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/NoCrypt/blessed_vae/resolve/main/blessed2.vae.pt -d -d {root}/stable-diffusion-webui/models/VAE/ -o blessed2.vae.pt
  
def installxformers():
  #!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.16/xformers-0.0.16+814314d.d20230118-cp38-cp38-linux_x86_64.whl
  #%pip install --no-deps -q https://github.com/brian6091/xformers-wheels/releases/download/0.0.15.dev0%2B4c06c79/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl
  %pip install -q xformers

def updatePython():
  !python --version > /content/pyversion
  with open('/content/pyversion', 'r') as file:
      if '3.10' in file.read():
        print('Already python 3.10. Skip install.')
        return

  #install python 3.10
  !apt-get update -y
  !apt-get install python3.10

  #change alternatives
  !rm /usr/local/bin/python
  !rm /usr/local/bin/pip
  !sudo apt-get install python3.10-distutils
  !sudo update-alternatives --install /usr/local/bin/python python /usr/bin/python3.10 2
  !wget https://bootstrap.pypa.io/get-pip.py && python get-pip.py

def initSaveGoogleDriveModelOnly():
  # Use config file in google drive
  %cd {root}/stable-diffusion-webui
  if not os.path.exists(output_path + '/ui-config.json'):
    print("Create new ui-config.json file.")    
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/ui-config.json -O {output_path + '/ui-config.json'}
  if not os.path.exists(output_path + '/config.json'):
    print("Create new config.json file.")    
    !wget https://github.com/sagiodev/stablediffusion_webui/raw/master/config.json -O {output_path + '/config.json'}
  if not os.path.exists(output_path + '/cache.json'):
    print("Create new cache.json file.")
    !wget https://github.com/plucyvrz/stablediffusion_webui/raw/master/cache.json -O {output_path + '/cache.json'} 
  if not os.path.exists(output_path + '/user.css'):
    print("Create new user.css file.")
    !wget https://github.com/plucyvrz/stablediffusion_webui/raw/master/user.css -O {output_path + '/user.css'} 

  #!rm /content/stable-diffusion-webui/ui-config.json
  !ln -s {output_path + '/ui-config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/config.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/cache.json'} {root}/stable-diffusion-webui/
  !ln -s {output_path + '/user.css'} {root}/stable-diffusion-webui/
  !ln -s {output_path}/outputs
  
  # embeddings folder on Google Drive
  !mkdir -p {output_path}/embeddings  
  !rm -rf embeddings
  !ln -s {output_path}/embeddings

  # save parameter file in google drive
  if not os.path.exists(output_path + '/params.txt'):
    !touch {output_path + '/params.txt'}
  !ln -s {output_path}/params.txt

  # link all models in the models folder
  %cd {root}/stable-diffusion-webui/models/Stable-diffusion
  models_in_google_drive = glob.glob(output_path + '/models/*')
  print('Models in Google Drive: %s'%models_in_google_drive)
  for f in models_in_google_drive:
    !ln -s {f}

  # link all upscalers in the model folder
  !mkdir -p {root}/stable-diffusion-webui/models/ESRGAN
  %cd {root}/stable-diffusion-webui/models/ESRGAN
  upscalers_in_google_drive = glob.glob(output_path + '/ESRGAN/*')
  print('Upscalers in Google Drive: %s'%upscalers_in_google_drive)
  for f in upscalers_in_google_drive:
    !ln -s {f}

  # use lora model folder in google drive
  !mkdir -p {output_path}/Lora
  %cd {root}/stable-diffusion-webui/models
  !rm -rf Lora
  !ln -s {output_path}/Lora

  # use hypernetwork folder in google drive
  !mkdir -p {output_path}/hypernetworks
  %cd {root}/stable-diffusion-webui/models
  !rm -rf hypernetworks
  !ln -s {output_path}/hypernetworks

  # use VAE folder in google rive
  !mkdir -p {output_path}/VAE
  %cd {root}/stable-diffusion-webui/models
  !rm -rf VAE
  !ln -s {output_path}/VAE

  # use ScuNET Upscaler
  !mkdir -p {root}/stable-diffusion-webui/models/ScuNET
  upscaler_scunet = glob.glob(output_path + '/ScuNET/*')
  print('Upscaler ScuNET: %s'%upscaler_scunet)
  for f in upscaler_scunet:
    !ln -s {f}

  # use lora models for AdditionalNetwork
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-additional-networks/models/lora
  lora_in_google_drive = glob.glob(output_path + '/Lora/*')
  print('Lora ready to use: %s'%lora_in_google_drive)
  for f in lora_in_google_drive:
    !ln -s {f}


def installControlNet():
  print("Installing ControlNet extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Mikubill/sd-webui-controlnet.git
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet
  !pip install -r requirements.txt
  
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-controlnet/models
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_ip2p.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11e_sd15_shuffle.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1e_sd15_tile.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_inpaint.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_lineart.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_mlsd.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_scribble.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_seg.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_softedge.pth')
  downloadModel('https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15s2_lineart_anime.pth')
  print("ControlNet install completed.")

def installTagComplete():
  print("Installing TagComplate extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/DominikDoom/a1111-sd-webui-tagcomplete
  print("TagComplete install completed.")

def installImageBrowser():
  print("Installing imageBrowser extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/yfszzx/stable-diffusion-webui-images-browser
  print("imageBrowser install completed.")

def installDarkTheme():
  print("Installing darkTheme extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/Gerschel/sd-web-ui-quickcss
  print("darkTheme install completed.")

def installInfiniteZoom():
  print("Installing infiniteZoom extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/v8hid/infinite-zoom-automatic1111-webui
  print("infiniteZoom install completed.")

def installAdditionalNetwork():
  print("Installing AdditionalNetwork extension...")
  %cd {root}/stable-diffusion-webui/extensions
  !git clone https://github.com/kohya-ss/sd-webui-additional-networks
  %cd {root}/stable-diffusion-webui/extensions/sd-webui-additional-networks
  !wget https://raw.githubusercontent.com/plucyvrz/stablediffusion_webui/master/hashes.json -O hashes.json
  print("AdditionalNetwork install completed.")

def installCivitAIBrowser():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone -b v2.1 https://github.com/camenduru/sd-civitai-browser

def installRatioHelper():
  %cd {root}/stable-diffusion-webui/extensions
  !git clone -b v2.1 https://github.com/camenduru/sd-webui-aspect-ratio-helper

def installDeforum():
  !git clone https://github.com/deforum-art/deforum-for-automatic1111-webui {root}/stable-diffusion-webui/extensions/deforum

def installRegionalPrompter():
  !git clone https://github.com/hako-mikan/sd-webui-regional-prompter {root}/stable-diffusion-webui/extensions/sd-webui-regional-prompter


def applyGitFilemode():
  # git default file mode prevents checkout and fail
  print('Apply git filemode false')
  !cd {root}/stable-diffusion-webui/repositories/k-diffusion;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/taming-transformers;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/CodeFormer;git config core.filemode false
  !cd {root}/stable-diffusion-webui/repositories/BLIP;git config core.filemode false  
  
def cloneRepositories():
  !git clone https://github.com/crowsonkb/k-diffusion.git {root}/stable-diffusion-webui/repositories/k-diffusion
  !git clone https://github.com/Stability-AI/stablediffusion.git {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai
  !git clone https://github.com/CompVis/taming-transformers.git {root}/stable-diffusion-webui/repositories/taming-transformers
  !git clone https://github.com/sczhou/CodeFormer.git {root}/stable-diffusion-webui/repositories/CodeFormer
  !git clone https://github.com/salesforce/BLIP.git {root}/stable-diffusion-webui/repositories/BLIP
  

def lowRamPatch():
  !sed -i -e '''/prepare_environment()/a\    os.system\(f\"""sed -i -e ''\"s/dict()))/dict())).cuda()/g\"'' {root}/stable-diffusion-webui/repositories/stable-diffusion-stability-ai/ldm/util.py""")''' {root}/stable-diffusion-webui/launch.py

def deleteRepos():
  # delete repository directories in webui
  !rm -rf {root}/stable-diffusion-webui/repositories

updatePython()

!mkdir -p {root}
os.chdir(root)
!apt-get -y install -qq aria2
!pip install pyngrok
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui



if gMode == GDriveSaveMode.Everything:
  # delete existing repositories and reclone so the file mode fix can be applied
  # otherwise some will only be cloned in the final launch, causing some to fail to checkout.
  deleteRepos()
  cloneRepositories()
  applyGitFilemode()

# fix torch, torchvision version mismatch error
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117


# A1111 first launch
%cd {root}/stable-diffusion-webui
!git checkout -f a9fed7c364061ae6efb37f797b6b522cb3cf7aa2
#!git checkout -f 22bcc7be428c94e9408f589966c2040187245d81
!COMMANDLINE_ARGS="--exit"  python launch.py

if AdditionalNetwork:
  installAdditionalNetwork()

if gMode == GDriveSaveMode.Models_only:
  initSaveGoogleDriveModelOnly()

download_models()

if ControlNet:
  installControlNet()

if Deforum:
  installDeforum()

if TagComplete:
  installTagComplete()

if imageBrowser:
  installImageBrowser()

if darkTheme:
  installDarkTheme()

if infiniteZoom:
  installInfiniteZoom()

if RatioHelper:
  installRatioHelper()

if CivitAIBrowser:
  installCivitAIBrowser()

if Regional_Prompter:
  installRegionalPrompter()


# clear output
if Clear_Log:
  clear()

if  gMode != GDriveSaveMode.Everything:
  lowRamPatch()

%cd {root}/stable-diffusion-webui  
args = f'--gradio-img2img-tool color-sketch --enable-insecure-extension-access --gradio-queue --xformers --theme=dark'

if NGROK:
  args += f' --ngrok {NGROK} '
else:
  args += ' --share '
if username and password:
  args += f' --gradio-auth {username}:{password} '
print(f'WEBUI ARGUMENTS: {args}')
!python {root}/stable-diffusion-webui/launch.py {args}